In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn")

In [5]:
class IterativeBase():

    def __init__(self, ticker, start, end, amount, use_spread=True):
        self.ticker = ticker
        self.start = start
        self.end = end
        self.initial_balance = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
        self.position = 0
        self.use_spread = use_spread
        self.get_data()

    def get_data(self):
        raw = pd.read_csv("/Users/maxibertonalbornoz/Documents/Python/Udemy/Algorithmic Trading A-Z with Python and ML/Part3_Materials/detailed.csv", parse_dates=["time"], index_col="time")
        raw = raw.loc[self.start:self.end]
        raw["returns"] = np.log(raw.price.div(raw.price.shift(periods=1)))
        
        self.data = raw

# Utilities
    def plot_data(self, cols=None):
        if cols is None:
            cols = "price"
        self.data[cols].plot(figsize=(15,8), title=self.ticker)

    def get_values(self, bar):
        date = str(self.data.index[bar].date())
        price = round(self.data.price.iloc[bar],5)
        spread = round(self.data.spread.iloc[bar],5)
        return date, price, spread

# Check status part
    def print_current_balance(self, bar):
        date, price, spread = self.get_values(bar)
        print("{} | Current Balance: {}".format(date, round(self.current_balance, 2)))

    def print_current_position_value(self, bar):
        date, price, spread = self.get_values(bar)
        cpv = self.units * price
        print("{} | Current Position Value: {}".format(date, round(cpv,2)))

    def print_current_nav(self, bar):
        date, price, spread = self.get_values(bar)
        nav = self.current_balance + self.units * price
        print("{} | Net Asset Value: {}".format(date, round(nav,2)))

# Buy and sell parts
    def buy_instrument(self, bar, units=None, amount=None):
        date, price, spread = self.get_values(bar)
        if self.use_spread == True:
            price += spread/2 # ask price
        if amount is not None:
            units = int(amount/price)
        self.current_balance -= units * price
        self.units += units
        self.trades += 1
        print("{} | Buying {} units for {}".format(date, units, round(price,5)))

    def sell_instrument(self, bar, units=None, amount=None):
        date, price, spread = self.get_values(bar)
        if self.use_spread == True:
            price -= spread/2 # bid price
        if amount is not None:
            units = int(amount/price)
        self.current_balance += units * price
        self.units -= units
        self.trades += 1
        print("{} | Selling {} units for {}".format(date, units, round(price,5)))

    def close_positions(self, bar):
        date, price, spread = self.get_values(bar)
        print(60*"-")
        print(" +++ CLOSING ALL POSITIONS +++ ")
        self.current_balance += self.units * price
        self.current_balance -= (abs(self.units)*spread/2*self.use_spread)
        self.units = 0
        self.trades += 1
        print("Closing position of {} units for {}".format(self.units, price))
        self.print_current_balance(bar)
        perf = (self.current_balance - self.initial_balance) / self.initial_balance * 100
        print("Performance in %: {}".format(perf))
        print("Number of trades executed: {}".format(self.trades))
        print(60*"-")

In [6]:
class IterativeBacktest(IterativeBase):

    def go_long(self, bar, units=None, amount=None):
        if self.position == -1:
            self.buy_instrument(bar, units=-self.units) # close current short position
        if units:
            self.buy_instrument(bar, units=units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(bar, amount=amount) # go long

    def go_short(self, bar, units=None, amount=None):
        if self.position == 1:
            self.sell_instrument(bar, units=self.units) # close current long position
        if units:
            self.sell_instrument(bar, units=units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(bar, amount=amount) # go short

# SMA Strategy
    def test_sma_strategy(self, sma_S, sma_L):
        
        # print a statement
        stm = "Testing SMA strategy for {} | SMA_S: {} | SMA_L: {}".format(self.ticker, sma_S, sma_L)
        print(65*"-")
        print(stm)
        print(65*"-")

        # reset everything
        self.position = 0 # initial neutral position
        self.get_data() # reset dataset
        self.current_balance = self.initial_balance # reset initial balance
        self.trades = 0 # reset trades

        # data features
        self.data["sma_S"] = self.data.price.rolling(window=sma_S).mean()
        self.data["sma_L"] = self.data.price.rolling(window=sma_L).mean()
        self.data.dropna(inplace=True)

        # strategy
        for bar in range(len(self.data)-1):
            if self.data["sma_S"].iloc[bar] > self.data["sma_L"].iloc[bar]:
                if self.position in [0, -1]:
                    self.go_long(bar, amount="all")
                    self.position = 1
            if self.data["sma_S"].iloc[bar] < self.data["sma_L"].iloc[bar]:
                if self.position in [0, 1]:
                    self.go_short(bar, amount="all")
                    self.position = -1
        self.close_positions(bar+1)

# Bollinger Bands Strategy
    def test_bollinger_strategy(self, sma=1, dev=1):

        # print a statement
        stm = "Testing Bollinger Band strategy for {} | SMA: {} and {} StdDev".format(self.ticker, sma, dev)
        print(65*"-")
        print(stm)
        print(65*"-")

        # reset everything
        self.position = 0 # initial neutral position
        self.get_data() # reset dataset
        self.current_balance = self.initial_balance # reset initial balance
        self.trades = 0 # reset trades

        # data features
        self.data["sma"] = self.data["price"].rolling(sma).mean()
        self.data["Upper"] = self.data["sma"] + self.data["price"].rolling(sma).std().mul(dev)
        self.data["Lower"] = self.data["sma"] - self.data["price"].rolling(sma).std().mul(dev)
        self.data.dropna(inplace=True)

        for bar in range(len(self.data)-1):
            if self.position == 0:
                if self.data["price"].iloc[bar] > self.data["Upper"].iloc[bar]:
                    self.go_short(bar, amount="all")
                    position = -1
                if self.data["price"].iloc[bar] < self.data["Lower"].iloc[bar]:
                    self.go_long(bar, amount="all")
                    position = 1
            
            elif self.position == 1:
                if self.data["price"].iloc[bar] > self.data["sma"].iloc[bar]:
                    if self.data["price"].iloc[bar] > self.data["Upper"].iloc[bar]:
                        self.go_short(bar, amount="all")
                        self.position = -1
                    else:
                        self.sell_instrument(bar, units=self.units)
                        self.position = 0
            
            elif self.position == -1:
                if self.data["price"].iloc[bar] < self.data["sma"].iloc[bar]:
                    if self.data["price"].iloc[bar] < self.data["Lower"].iloc[bar]:
                        self.go_long(bar, amount="all")
                        self.position = 1
                    else:
                        self.buy_instrument(bar, units= -self.units)
                        self.position = 0
        self.close_positions(bar+1)

# Contrarian Strategy
    def test_contrarian_strategy(self, window=1):
        self.window=window

                # print a statement
        stm = "Testing Contrarian strategy for {} | Window: {}".format(self.ticker, self.window)
        print(65*"-")
        print(stm)
        print(65*"-")

        # reset everything
        self.position = 0 # initial neutral position
        self.get_data() # reset dataset
        self.current_balance = self.initial_balance # reset initial balance
        self.trades = 0 # reset trades

        # data features
        self.data["rolling_returns"] = self.data.returns.rolling(window=self.window).mean()
        self.data.dropna(inplace=True)

        # strategy
        for bar in range(len(self.data)-1):
            if self.data.rolling_returns.iloc[bar] > 0: # go short
                if self.position in [0, 1]:    
                    self.go_short(bar, amount="all")
                    self.position = -1
            elif self.data.rolling_returns.iloc[bar] <= 0: # go long
                if self.position in [0, -1]:
                    self.go_long(bar, amount="all")
                    self.position = 1
        self.close_positions(bar+1)

In [7]:
bc = IterativeBacktest("EURUSD", "2006-12-31", "2020-06-30", 100000, use_spread=False)

In [10]:
bc.test_bollinger_strategy(sma=10, dev=2)

-----------------------------------------------------------------
Testing Bollinger Band strategy for EURUSD | SMA: 10 and 2 StdDev
-----------------------------------------------------------------
2007-01-22 | Selling 76766 units for 1.30265
2007-02-13 | Selling 152311 units for 1.31309
2007-02-26 | Selling 302062 units for 1.32422
2007-03-15 | Selling 600690 units for 1.33179
2007-04-04 | Selling 1191370 units for 1.34298
2007-05-02 | Buying 2361603 units for 1.355
2007-06-03 | Selling 0 units for 1.34892
2007-06-04 | Selling 0 units for 1.35243
2007-06-28 | Selling 0 units for 1.3541
2007-07-01 | Selling 0 units for 1.36235
2007-07-09 | Selling 0 units for 1.37472
2007-07-24 | Buying 0 units for 1.37212
2007-07-26 | Buying 0 units for 1.36352
2007-08-14 | Buying 0 units for 1.34425
2007-08-23 | Selling 0 units for 1.36754
2007-09-06 | Selling 0 units for 1.3768
2007-09-19 | Selling 0 units for 1.40646
2007-10-17 | Selling 0 units for 1.42934
2007-11-06 | Selling 0 units for 1.46372


In [11]:
bc.test_contrarian_strategy(window=3)

ing 104833 units for 1.06444
2017-04-11 | Selling 104833 units for 1.06655
2017-04-11 | Selling 104833 units for 1.06655
2017-04-16 | Buying 104833 units for 1.06434
2017-04-16 | Buying 105269 units for 1.06434
2017-04-17 | Selling 105269 units for 1.07299
2017-04-17 | Selling 105269 units for 1.07299
2017-04-20 | Buying 105269 units for 1.07268
2017-04-20 | Buying 105329 units for 1.07268
2017-04-23 | Selling 105329 units for 1.08674
2017-04-23 | Selling 105329 units for 1.08674
2017-04-27 | Buying 105329 units for 1.08962
2017-04-27 | Buying 104773 units for 1.08962
2017-05-01 | Selling 104773 units for 1.09309
2017-05-01 | Selling 104773 units for 1.09309
2017-05-02 | Buying 104773 units for 1.08852
2017-05-02 | Buying 105652 units for 1.08852
2017-05-03 | Selling 105652 units for 1.09842
2017-05-03 | Selling 105652 units for 1.09842
2017-05-08 | Buying 105652 units for 1.08743
2017-05-08 | Buying 107788 units for 1.08743
2017-05-11 | Selling 107788 units for 1.09314
2017-05-11 | Se

In [21]:
bc = InterativeBacktest("EURUSD", "2006-12-31", "2020-06-30", 100000, use_spread=True)

In [22]:
bc.test_bollinger_strategy(sma=30, dev=2)

-----------------------------------------------------------------
Testing Bollinger Band strategy for EURUSD | SMA: 30 and 2 StdDev
-----------------------------------------------------------------
2007-02-13 | Selling 76159 units for 1.31303
2007-02-14 | Selling 152190 units for 1.31414
2007-02-15 | Selling 304570 units for 1.31332
2007-02-17 | Selling 608260 units for 1.31522
2007-02-18 | Selling 1216224 units for 1.31554
2007-03-15 | Selling 2403670 units for 1.33129
2007-03-16 | Selling 4807340 units for 1.33129
2007-03-17 | Selling 9616053 units for 1.3311
2007-03-20 | Selling 19125922 units for 1.33849
2007-04-11 | Selling 37980855 units for 1.34804
2007-04-12 | Selling 75725216 units for 1.35225
2007-04-14 | Selling 151159794 units for 1.35485
2007-04-15 | Selling 302641249 units for 1.35341
2007-04-16 | Selling 603899261 units for 1.35651
2007-04-17 | Selling 1203982020 units for 1.36081
2007-04-18 | Selling 2407380244 units for 1.36114
2007-05-21 | Buying 4872827447 units for 

In [23]:
bc.test_contrarian_strategy(window=3)

017-03-26 | Selling 78609 units for 1.08618
2017-03-26 | Selling 78609 units for 1.08618
2017-03-28 | Buying 78609 units for 1.0767
2017-03-28 | Buying 79994 units for 1.0767
2017-04-04 | Selling 79994 units for 1.06625
2017-04-04 | Selling 79994 units for 1.06625
2017-04-05 | Buying 79994 units for 1.06452
2017-04-05 | Buying 80254 units for 1.06452
2017-04-11 | Selling 80254 units for 1.06645
2017-04-11 | Selling 80254 units for 1.06645
2017-04-16 | Buying 80254 units for 1.06457
2017-04-16 | Buying 80536 units for 1.06457
2017-04-17 | Selling 80536 units for 1.0729
2017-04-17 | Selling 80536 units for 1.0729
2017-04-20 | Buying 80536 units for 1.07318
2017-04-20 | Buying 80494 units for 1.07318
2017-04-23 | Selling 80494 units for 1.08659
2017-04-23 | Selling 80494 units for 1.08659
2017-04-27 | Buying 80494 units for 1.09012
2017-04-27 | Buying 79972 units for 1.09012
2017-05-01 | Selling 79972 units for 1.09294
2017-05-01 | Selling 79972 units for 1.09294
2017-05-02 | Buying 79972